In [1]:
import numpy as np
from tqdm import tqdm

data = np.load('./data/eastmoney_full_stocks_list_nlu_tencent.pkl', allow_pickle=True)

In [2]:
data[0]

{'ticker_id': '603987',
 'ticker_name': '康德莱',
 'baike_summary': '浙江康德莱医疗器械股份有限公司成立于1987年，是康德莱集团属下一家专业生产一次性针类系列医疗器械的制造厂家。主要是生产加工,经销批发医疗器械。',
 'baike_content': '浙江康德莱医疗器械股份有限公司成立于1987年，系康德莱集团属下一家专业生产一次性针类系列医疗器械的制造厂家。2009年度公司实现产值26656万元，出口创汇超6220万元，实现利税4848万元。公司拥有十万级净化车间七千多平方米,是浙江省“高新技术企业”，浙江省AAA信用单位；公司生产的“一次性使用采血针”获得了省高新技术产品称号，“一次性使用无菌注射针”获得浙江省名牌产品称号。公司现具有年产60亿支各类医用针尖、30亿支一次性使用注射针的能力，为国内同行前列。公司以科学的管理、先进的技术、齐全的品种、高档的质量、优质的服务赢得了国内外广大用户的青睐，于1998年在同行业首家通过CMDC认证后，又先后通欧盟TUV和美国FDA工厂审核。产品不仅销往全国各地，还远销美国、欧洲、南美、非洲、中东、东南亚等数十个国家和地区，出口达30多个品种，在国际市场占据了一定的份额。康德莱集团现担任中国医疗器械协会高分子分会理事长单位，总裁张宪淼先生是医疗器械高分子分会理事长，浙江康德莱股份公司担任中国注射器、针标准化技术委员会主任单位。现行的 “制造医疗器械用不锈钢针管”、“一次性使用血样采集针”、“一次性使用牙科注射针”、“一次性使用静脉输液针”等国家标准和行业标准均由公司承担起草。康德莱正以共生理念为指导，以“创造KDL名牌，取信亿万用户”为质量方针，以造福人类为宗旨，以满足客户的需求为目标而不懈追求，努力挤身全球医用针制造行业三强之列。2020年1月20日，评为2019 年度温州市市长质量奖企业。[1]2021年10月，入选2021年浙江省省级新一代信息技术与制造业融合发展试点示范企业名单（网络协同制造方向）。[2]',
 'baike_url': 'https://baike.baidu.com/item/康德莱',
 'text_keyword': [{"Score": 1.0, "Word": "公司"},
  {"Score": 0.88982016, 

In [3]:
# use tecent to analyze

import pandas as pd
from tqdm import tqdm
from utils.tencent_api import TencentNLU
import pickle

config = {
    'SecretId': 'AKIDlMhlY5LK2TdXBYfV4MO9droVv4vWrugB',
    'SecretKey': 'GFxOYS7VdT2KM8cpqa3E8l7C0Zsue9OL',
}

nlu_tool = TencentNLU(config)

In [4]:
all = []

for item in tqdm(data):

    try:
        title = item['baike_summary']
        content = item['baike_content']
        input = (title + content)[0:400]
        resp = nlu_tool.get_text_vector(input)

        one = {
            'ticker_id': item['ticker_id'],
            'ticker_name': item['ticker_name'],
            'vector': np.array(resp.Vector),
        }

        all.append(one)

    except Exception:
        pass

with open('./data_ignore/vectors_tencent_api.pkl','wb') as f:
    pickle.dump(all, f)

100%|██████████| 510/510 [02:05<00:00,  4.07it/s]


In [5]:
data = np.load('./data_ignore/vectors_tencent_api.pkl', allow_pickle=True)

In [6]:
ticker_names = []
ticker_ids = []
vectors = []

for idx in tqdm(range(len(data))):
    if 'vector' in data[idx].keys():
        vectors.append(data[idx]['vector'])
        ticker_ids.append(data[idx]['ticker_id'])
        ticker_names.append(data[idx]['ticker_name'])


100%|██████████| 510/510 [00:00<00:00, 505337.83it/s]


In [7]:
vectors = np.stack(vectors)

In [8]:
vectors.shape

(510, 768)

In [9]:
def similarity_vector_matrix(arr, brr):
    return arr.dot(brr.T) / (np.sqrt(np.sum(arr*arr)) * np.sqrt(np.sum(brr*brr, axis=1)))

In [10]:
def stock_search(query, topk=10):

    # query = '医疗保险，重大疾病保障'

    resp = nlu_tool.get_text_vector(query)
    vector = np.array(resp.Vector)

    res = similarity_vector_matrix(vector, vectors)
    idxs = np.argsort(res)[::-1]

    topk_idxs = idxs[:10]
    names = [ticker_names[idx] for idx in topk_idxs]
    print(names)

- 自动驾驶，新能源汽车
- 电影，电视剧，文化艺术
- 啤酒，烧烤，朋友聚会
- 医疗保险，重大疾病保障
- 新冠肺炎

In [11]:
query = '自动驾驶，新能源汽车'
stock_search(query)

['杉杉股份', '新希望', '阿尔特', '新大陆', '宝钢股份', '高德红外', '中国能建', '新经典', '千方科技', '三安光电']


In [12]:
query = '电影，电视剧，文化艺术'
stock_search(query)

['新华网', '中国核电', '京沪高铁', '中海油服', '片仔癀', '伯特利', '金地集团', '新大陆', '金徽酒', '新华医疗']


In [13]:
query = '啤酒，烧烤，朋友聚会'
stock_search(query)

['金徽酒', '中国长城', '中国石油', '汉钟精机', '四川路桥', '高德红外', '双汇发展', '东方集团', '中科电气', '长城汽车']


In [14]:
query = '医疗保险，重大疾病保障'
stock_search(query)

['宁波银行', '健康元', '贵州茅台', '重庆啤酒', '爱尔眼科', '中青旅', '今世缘', '滨江集团', '中航重机', '绿色动力']


In [15]:
query = '新冠肺炎'
stock_search(query)

['华能国际', '中国化学', '中青旅', '万华化学', '振华科技', '步步高', '中国联通', '华鲁恒升', '奥佳华', '贵州茅台']
